In [41]:
import heapq

import mglearn as mglearn
import np as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import sklearn_evaluation
from sklearn.feature_selection import SelectFromModel, SelectPercentile, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.svm import SVC, LinearSVC

!pip install numpy scipy scikit-learn matplotlib pandas
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.ensemble import StackingClassifier
import heapq


You should consider upgrading via the '/Users/engineer/workspace/cse590-machine-learning/venv/bin/python -m pip install --upgrade pip' command.


In [67]:

X = pd.read_csv("../dataset/extracted_features.csv").values
index_for_removal = np.array(
    [
        0, 10, 14, 17, 25, 26, 35, 39, 49, 59, 62,
    ]
)
X_indexed = np.delete(X, index_for_removal, 1)

y = pd.read_csv("../dataset/labels.csv").values
# images = pd.read_csv("../dataset/raw_images.csv").values
X_train, X_test, y_train, y_test = train_test_split(
    X_indexed, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
)
y_train = y_train.ravel()

select = SelectPercentile(percentile=90)
select.fit(X_train, y_train)
X_train = select.transform(X_train)
X_test = select.transform(X_test)

rf = RandomForestClassifier(
    n_estimators=200,
    max_features=3,
).fit(X_train, y_train)

select_sfm = SelectFromModel(
    rf,
    threshold='0.1*mean',
)
select_sfm.fit(X_train, y_train)
X_train = select_sfm.transform(X_train)
X_test = select_sfm.transform(X_test)


select_rfe = RFE(RandomForestClassifier(
    n_estimators=200, random_state=42
), n_features_to_select=40)

select_rfe.fit(X_train, y_train)
X_train = select_rfe.transform(X_train)
X_test = select_rfe.transform(X_test)


scaler_standard = StandardScaler().fit(X_train)
X_train_standard = scaler_standard.transform(X_train)
X_test_standard = scaler_standard.transform(X_test)

scaler_robust = RobustScaler().fit(X_train)
X_train_robust = scaler_robust.transform(X_train)
X_test_robust = scaler_robust.transform(X_test)

scaler_minmax = MinMaxScaler().fit(X_train)
X_train_minmax = scaler_minmax.transform(X_train)
X_test_minmax = scaler_minmax.transform(X_test)
#
# pca = PCA(n_components=56, random_state=42)
# X_train_pca = pca.fit_transform(X_train_standard)
# X_test_pca = pca.transform(X_test_standard)

In [7]:

knn_pipeline = make_pipeline(
    MinMaxScaler(),
    KNeighborsClassifier(
        n_neighbors=5,
    )
)
ksvm_pipeline = make_pipeline(
    MinMaxScaler(),
    SVC(
        kernel='poly',
        C=5,
        degree=1,
        random_state=42
    )
)
logreg_pipeline = make_pipeline(
    MinMaxScaler(),
    LogisticRegression(
        C=4.817948717948719
    )
)
lsvc_pipeline = make_pipeline(
    MinMaxScaler(),
    LinearSVC(
        C=1.6326530612244898
    )
)
mnb_pipeline = make_pipeline(
    MinMaxScaler(),
    MultinomialNB(
        alpha=0.15789473684210525,
    )
)
rf_pipeline = make_pipeline(
    RandomForestClassifier(
        n_estimators=500,
        max_features=5,
        random_state=42
    )
)
gbrt_pipeline = make_pipeline(
    RobustScaler(),
    GradientBoostingClassifier(
        n_estimators=500,
        max_depth=1,
        learning_rate=0.1,
        random_state=42
    )
)
mlp_pipeline = make_pipeline(
    MinMaxScaler(),
    MLPClassifier(
        activation='relu',
        hidden_layer_sizes=256,
        alpha=0.05,
        random_state=42
    )
)


In [10]:

estimators = [
    # ("ksvm", ksvm_pipeline),
    ("mlp", mlp_pipeline),
    ("gbrt", gbrt_pipeline),
    # ("rf", rf_pipeline),
    # ("mnb", mnb_pipeline),
    # ("knn", knn_pipeline),
    ("lsvc", lsvc_pipeline),
    ("mnb", mnb_pipeline),
    # ("logreg", logreg_pipeline),
]

stacking_classifier = StackingClassifier(
    estimators=estimators,
    final_estimator=logreg_pipeline
)

In [68]:
stacking_classifier.fit(X_train, y_train)
print(stacking_classifier.score(X_train, y_train))
print(stacking_classifier.score(X_test, y_test))

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maxim

0.9819718309859155
0.9414414414414415


/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
voting_classifier = VotingClassifier(
    estimators=estimators,
)
voting_classifier.fit(X_train, y_train)
print(voting_classifier.score(X_train, y_train))
print(voting_classifier.score(X_test, y_test))

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.8969014084507042
0.8558558558558559


In [14]:
rf_minmax = RandomForestClassifier(
    n_estimators=500,
    max_features=5,
).fit(X_train_minmax, y_train)


In [ ]:

select_sfm = SelectFromModel(
    rf_minmax,
    threshold='median'
)

select_sfm.fit(X_train_minmax, y_train)
X_train_select_rf = select_sfm.transform(X_train_minmax)
X_test_select_rf = select_sfm.transform(X_test_minmax)


In [16]:
stacking_classifier.fit(X_train_select_rf, y_train)
print(stacking_classifier.score(X_train_select_rf, y_train))
print(stacking_classifier.score(X_test_select_rf, y_test))

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maxim

0.9752112676056338


ValueError: X has 64 features, but MinMaxScaler is expecting 32 features as input.

In [63]:
print(stacking_classifier.score(X_test, y_test))
y_pred = stacking_classifier.predict(X_test)
y_pred_proba = stacking_classifier.predict_proba(X_test)
print(y_pred.shape)
print(y_pred_proba.shape)

0.9414414414414415
(444,)
(444, 9)


In [64]:
pd.DataFrame({'pred': y_pred, 'proba': y_pred_proba[:,1]})

,pred,proba
0,0,0.000002
1,0,0.000075
2,3,0.000248
3,2,0.001843
4,2,0.003825
...,...,...
439,4,0.000156
440,7,0.018201
441,8,0.035370
442,5,0.000077
